In [1]:
from kaggle_environments import make
import numpy as np
import gym
import torch
from Scripts.MCTS import agent_mcts
from Scripts.Deep_Q_Learning import DQN
import matplotlib.pyplot as plt

Loading environment lux_ai_s2 failed: No module named 'vec_noise'


c:\Users\Elian Morel\.conda\envs\tetris\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cols = 7
rows = 6
env = make("connectx", configuration={"rows":rows, "columns":cols})
sync_freq = 10

In [3]:
EPISODES = 100
observation_space = gym.spaces.Discrete(cols * rows).n
action_space = gym.spaces.Discrete(cols)

In [11]:
def dqn_agent(obs,config):
    global load_network
    agent =DQN()
    agent.network = load_network
    print(load_network)
    return agent.choose_action(obs)
    

In [27]:
trainer = env.train([None, agent_mcts])
agent =DQN()
best_reward = 0
average_reward = 0
episode_number = []
average_reward_number = []
load_network = 0
j=0
flag = False
EPISODES = 5000
RANDOM_PLAYS = 1000
UPDATE_NETWORK = 1000
for i in range(1, EPISODES):
    state, info = env.reset()
    state = state["observation"]["board"]
    state = np.reshape(state, [1, rows, cols])
    score = 0
    if i < RANDOM_PLAYS:
        trainer = env.train([None,"random"])
        
    else: 
        if(i %UPDATE_NETWORK == 0):
            print("aaa")
            load_network = agent.network
            trainer = env.train([None,dqn_agent])
            agent.exploration_rate = 0.5
            plt.plot(episode_number[-1000:],average_reward_number[-1000:])
    if i%10000 == 0:
        torch.save({'model_state_dict': agent.network.state_dict(),
        'optimizer_state_dict': agent.network.optimizer.state_dict(),
        'epoch': EPISODES},"Reinforced-Connect4/models/network" + str(i) + ".pkl"
        )
    while True:
        #print(j)
        j+=1

        action = agent.choose_action(state)
        state_, reward, done, info = trainer.step(action)
        if reward == None :
            reward = -100
        state_ = state_["board"]
        state_ = np.reshape(state_, [1, rows, cols])
        state = torch.tensor(state).float()
        state_ = torch.tensor(state_).float()

        #print("state1 : ", state)
        #print("state2 : ", state_)
        exp = (state, action, reward, state_, done)
        agent.replay.add(exp)
        agent.learn()

        state = state_
        score += reward

        if j % sync_freq == 0:
            agent.network2.load_state_dict(agent.network.state_dict())

        if done:
            if score > best_reward:
                best_reward = score
            average_reward += score 
            if i%10==0:
                print("Episode {} Average Reward {} Best Reward {} Last Reward {} Epsilon {}".format(i, average_reward/i, best_reward, score, agent.returning_epsilon()))
                #test_model(agent,10, observation_space)
            break
  
        episode_number.append(i)
        average_reward_number.append(average_reward/i)

plt.plot(episode_number, average_reward_number)
plt.show()

C:\Users\Elian Morel\AppData\Local\Temp\ipykernel_23468\1774338429.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state).float()


Episode 10 Average Reward -19.6 Best Reward 1 Last Reward 1 Epsilon 0.9531108968798944


c:\05-Elian\04-Polytechnique\03-3A\10- Advanced Machine Learning\projet\Scripts\Deep_Q_Learning.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(observation).float().detach()


Episode 20 Average Reward -14.95 Best Reward 1 Last Reward -1 Epsilon 0.8658253647948594
Episode 30 Average Reward -29.9 Best Reward 1 Last Reward -100 Epsilon 0.7833919908382508
Episode 40 Average Reward -29.9 Best Reward 1 Last Reward -100 Epsilon 0.7173681503955072
Episode 50 Average Reward -27.84 Best Reward 1 Last Reward 1 Epsilon 0.6635141250307047
Episode 60 Average Reward -31.55 Best Reward 1 Last Reward -1 Epsilon 0.6009433131399452
Episode 70 Average Reward -32.75714285714286 Best Reward 1 Last Reward 1 Epsilon 0.5448178815347047
Episode 80 Average Reward -31.1375 Best Reward 1 Last Reward 1 Epsilon 0.48901714023696286
Episode 90 Average Reward -30.977777777777778 Best Reward 1 Last Reward -1 Epsilon 0.4514016473420717
Episode 100 Average Reward -28.83 Best Reward 1 Last Reward 1 Epsilon 0.41584661836923925
Episode 110 Average Reward -27.990909090909092 Best Reward 1 Last Reward -100 Epsilon 0.3827090189332178
Episode 120 Average Reward -28.133333333333333 Best Reward 1 Last 

In [34]:
trainer = env.train([None,agent_mcts])

In [21]:
agent.network

NetworkB(
  (conv_layer): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1536, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=7, bias=True)
  )
  (loss): MSELoss()
)

In [26]:
nb_games = 0
history = {1:0, -1:0,0:0, None:0}
env.reset()
trainer = env.train([None,dqn_agent])
while nb_games < 200:
    #env.render()
    
    if done :
        #print(np.reshape(observation["board"], (6,7)))
        observation, info = env.reset()
        observation = observation["observation"]
        history[reward] += 1
        nb_games +=1
        
    
    action = agent.choose_action(np.reshape(observation["board"], [1, rows,cols]))
    observation, reward, done, info = trainer.step(action)

In [24]:
history

{1: 16, -1: 22, None: 162}

In [33]:
agent.network.

<bound method Module.state_dict of Network(
  (fc1): Linear(in_features=42, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=7, bias=True)
  (loss): MSELoss()
)>